### 1. Import modules

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pyodbc
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler

### 2. Merge .pkl's

In [2]:
# schoolgaanden
schoolgaanden_observaties = pd.read_pickle('4_Data\Pickles\schoolgaanden_observaties.pkl')
schoolgaanden_relaties = pd.read_pickle('4_Data\Pickles\schoolgaanden_relaties.pkl')
schoolgaanden_toestemmingen = pd.read_pickle('4_Data\Pickles\schoolgaanden_toestemmingen.pkl')
schoolgaanden_cbs_pc4 = pd.read_pickle('4_Data\Pickles\schoolgaanden_cbs4.pkl')
schoolgaanden_cbs_buurt = pd.read_pickle('4_Data\Pickles\schoolgaanden_buurt.pkl')
schoolgaanden_coronit = pd.read_pickle('4_Data\Pickles\schoolgaanden_coronit.pkl')

In [3]:
# Merge schoolgaanden_observaties, schoolgaanden_PC6 and schoolgaanden_relaties on 'Clientnummer'
schoolgaanden = (schoolgaanden_observaties
                 .merge(schoolgaanden_toestemmingen, on='Clientnummer', how='left')
                 .merge(schoolgaanden_cbs_pc4, on='Clientnummer', how='left')
                 .merge(schoolgaanden_cbs_buurt, on='Clientnummer', how='left')
                 .merge(schoolgaanden_relaties, on='Clientnummer', how='left'))

In [4]:
len(schoolgaanden)

14047

#### 2.1 Parent 1

In [5]:
# Change the dtype of schoolgaanden['Huisnummer'] to int64
schoolgaanden['Huisnummer'] = schoolgaanden['Huisnummer'].astype('int64')

##### 2.1.1 Merge CoronIT on all four characteristics

In [6]:
# Merge schoolgaanden with df_coronit
schoolgaanden_coronit = schoolgaanden_coronit.rename(columns={'postcode': 'Postcode',
                                                              'huisnummer': 'Huisnummer',
                                                              'geboortedatum_ym': 'Geboortejaarmaand_relatie_ouder1',
                                                              'geslacht': 'Geslacht_relatie_ouder1',
                                                              'gevaccineerd': 'Gevaccineerd_ouder1'})
schoolgaanden = schoolgaanden.merge(schoolgaanden_coronit, on=['Postcode', 'Huisnummer', 'Geslacht_relatie_ouder1', 'Geboortejaarmaand_relatie_ouder1'], how='left')

In [7]:
# Print the missingness percentage of Gevaccineerd_ouder1
schoolgaanden['Gevaccineerd_ouder1'].isnull().mean()

0.31138321349754394

##### 2.1.2 Merge CoronIT after discarding sex (only for NA's after 2.1)

In [8]:
# Get the subset of schoolgaanden where Gevaccineerd_ouder1 is missing
missing_gevaccineerd = schoolgaanden[schoolgaanden['Gevaccineerd_ouder1'].isnull()]

# From schoolgaanden_coronit, remove columns 'Geslacht_relatie_ouder1'
schoolgaanden_coronit = schoolgaanden_coronit.drop(columns='Geslacht_relatie_ouder1')

# Merge again, but without Geslacht_relatie_ouder1
updated_missing = missing_gevaccineerd.merge(schoolgaanden_coronit, on=['Postcode', 'Huisnummer', 'Geboortejaarmaand_relatie_ouder1'], how='left')

# From updated_missing, remove columns 'Gevaccineerd_ouder1_x'
updated_missing = updated_missing.drop(columns='Gevaccineerd_ouder1_x')

# Concatenate the updated subset back with the non-missing part
non_missing = schoolgaanden[~schoolgaanden['Gevaccineerd_ouder1'].isnull()]
schoolgaanden = pd.concat([non_missing, updated_missing], ignore_index=True)

# Use combine_first to combine Gevaccineerd_ouder1 and Gevaccineerd_ouder1_y
schoolgaanden['Gevaccineerd_ouder1'] = schoolgaanden['Gevaccineerd_ouder1'].combine_first(schoolgaanden['Gevaccineerd_ouder1_y'])

# Drop Gevaccineerd_ouder1_y
schoolgaanden = schoolgaanden.drop('Gevaccineerd_ouder1_y', axis=1)

In [9]:
# Print the missingness percentage of Gevaccineerd_ouder1
schoolgaanden['Gevaccineerd_ouder1'].isnull().mean()

0.039581405282266675

In [10]:
schoolgaanden['Gevaccineerd_ouder1'].value_counts(normalize=True)

Gevaccineerd_ouder1
True     0.660663
False    0.339337
Name: proportion, dtype: float64

#### 2.2 Parent 2

In [11]:
schoolgaanden_coronit = pd.read_pickle('4_Data\Pickles\schoolgaanden_coronit.pkl')

##### 2.2.1 Merge CoronIT on all four characteristics

In [12]:
# Merge schoolgaanden with df_coronit
schoolgaanden_coronit = schoolgaanden_coronit.rename(columns={'postcode': 'Postcode',
                                                              'huisnummer': 'Huisnummer',
                                                              'geboortedatum_ym': 'Geboortejaarmaand_relatie_ouder2',
                                                              'geslacht': 'Geslacht_relatie_ouder2',
                                                              'gevaccineerd': 'Gevaccineerd_ouder2'})
schoolgaanden = schoolgaanden.merge(schoolgaanden_coronit, on=['Postcode', 'Huisnummer', 'Geslacht_relatie_ouder2', 'Geboortejaarmaand_relatie_ouder2'], how='left')

In [13]:
# Print the missingness percentage of Gevaccineerd_ouder1
schoolgaanden['Gevaccineerd_ouder2'].isnull().mean()

0.32362782088702213

##### 2.2.2 Merge CoronIT after discarding sex (only for NA's after 2.1)

In [14]:
# Get the subset of schoolgaanden where Gevaccineerd_ouder1 is missing
missing_gevaccineerd = schoolgaanden[schoolgaanden['Gevaccineerd_ouder2'].isnull()]

# From schoolgaanden_coronit, remove columns 'Geslacht_relatie_ouder1'
schoolgaanden_coronit = schoolgaanden_coronit.drop(columns='Geslacht_relatie_ouder2')

# Merge again, but without Geslacht_relatie_ouder1
updated_missing = missing_gevaccineerd.merge(schoolgaanden_coronit, on=['Postcode', 'Huisnummer', 'Geboortejaarmaand_relatie_ouder2'], how='left')

# From updated_missing, remove columns 'Gevaccineerd_ouder1_x'
updated_missing = updated_missing.drop(columns='Gevaccineerd_ouder2_x')

# Concatenate the updated subset back with the non-missing part
non_missing = schoolgaanden[~schoolgaanden['Gevaccineerd_ouder2'].isnull()]
schoolgaanden = pd.concat([non_missing, updated_missing], ignore_index=True)

# Use combine_first to combine Gevaccineerd_ouder1 and Gevaccineerd_ouder1_y
schoolgaanden['Gevaccineerd_ouder2'] = schoolgaanden['Gevaccineerd_ouder2'].combine_first(schoolgaanden['Gevaccineerd_ouder2_y'])

# Drop Gevaccineerd_ouder1_y
schoolgaanden = schoolgaanden.drop('Gevaccineerd_ouder2_y', axis=1)

In [15]:
# Print the missingness percentage of Gevaccineerd_ouder1
schoolgaanden['Gevaccineerd_ouder2'].isnull().mean()

0.05759236847725493

In [16]:
schoolgaanden['Gevaccineerd_ouder2'].value_counts(normalize=True)

Gevaccineerd_ouder2
True     0.620335
False    0.379665
Name: proportion, dtype: float64

### 3. Apply transformations

#### 3.1 Parent matching

In [17]:
# If both Geboortejaarmaand_relatie_ouder1 and GeboortejaarmaandBiologischeMoeder are not null and not equal, switch the columns
mask = ((schoolgaanden['Geboortejaarmaand_relatie_ouder1'].notnull()) &
        (schoolgaanden['GeboortejaarmaandBiologischeMoeder'].notnull()) &
        (schoolgaanden['Geboortejaarmaand_relatie_ouder1'] != schoolgaanden['GeboortejaarmaandBiologischeMoeder']) &
        (schoolgaanden['Geboortejaarmaand_relatie_ouder2'] == schoolgaanden['GeboortejaarmaandBiologischeMoeder']))

# Switch
schoolgaanden.loc[mask, ['Geboortejaarmaand_relatie_ouder1',
                       'Geboorteland_relatie_ouder1',
                       'Geslacht_relatie_ouder1',
                       'Geboortejaarmaand_relatie_ouder2',
                       'Geboorteland_relatie_ouder2',
                       'Geslacht_relatie_ouder2']] = schoolgaanden.loc[mask, ['Geboortejaarmaand_relatie_ouder2',
                                                                            'Geboorteland_relatie_ouder2',
                                                                            'Geslacht_relatie_ouder2',
                                                                            'Geboortejaarmaand_relatie_ouder1',
                                                                            'Geboorteland_relatie_ouder1',
                                                                            'Geslacht_relatie_ouder1']].values

In [18]:
print(f'The amount of relations in the df_observaties_relaties is: {len(schoolgaanden)}, the amount of unique client ID\'s is: {len(schoolgaanden['Clientnummer'].unique())}')

The amount of relations in the df_observaties_relaties is: 14047, the amount of unique client ID's is: 14047


In [19]:
# Set all 'onbekend' values in schoolgaanden['Geboortejaarmaand_relatie_ouder1'] to np.nan
schoolgaanden = schoolgaanden.replace('onbekend', np.nan)

# Print matches of parents
print(f'The amount of rows where both Geboortejaarmaand_relatie_ouder1 and GeboortejaarmaandBiologischeMoeder are empty: {len(schoolgaanden[(schoolgaanden['Geboortejaarmaand_relatie_ouder1'].isnull()) & (schoolgaanden['GeboortejaarmaandBiologischeMoeder'].isnull())])}')
print(f'The amount of rows where Geboortejaarmaand_relatie_ouder1 is empty and GeboortejaarmaandBiologischeMoeder is not: {len(schoolgaanden[(schoolgaanden['Geboortejaarmaand_relatie_ouder1'].isnull()) & (schoolgaanden['GeboortejaarmaandBiologischeMoeder'].notnull())])}')
print(f'The amount of rows where Geboortejaarmaand_relatie_ouder1 is not empty and GeboortejaarmaandBiologischeMoeder is empty: {len(schoolgaanden[(schoolgaanden['Geboortejaarmaand_relatie_ouder1'].notnull()) & (schoolgaanden['GeboortejaarmaandBiologischeMoeder'].isnull())])}')
print(f'The amount of rows where Geboortejaarmaand_relatie_ouder1 is not empty and GeboortejaarmaandBiologischeMoeder is not empty and they are not equal: {len(schoolgaanden[(schoolgaanden['Geboortejaarmaand_relatie_ouder1'].notnull()) & (schoolgaanden['GeboortejaarmaandBiologischeMoeder'].notnull()) & (schoolgaanden['Geboortejaarmaand_relatie_ouder1'] != schoolgaanden['GeboortejaarmaandBiologischeMoeder'])])}')
print(f'The amount of rows where Geboortejaarmaand_relatie_ouder1 and GeboortejaarmaandBiologischeMoeder are equal: {len(schoolgaanden[(schoolgaanden['Geboortejaarmaand_relatie_ouder1'].notnull()) & (schoolgaanden['GeboortejaarmaandBiologischeMoeder'].notnull()) & (schoolgaanden['Geboortejaarmaand_relatie_ouder1'] == schoolgaanden['GeboortejaarmaandBiologischeMoeder'])])}')

The amount of rows where both Geboortejaarmaand_relatie_ouder1 and GeboortejaarmaandBiologischeMoeder are empty: 217
The amount of rows where Geboortejaarmaand_relatie_ouder1 is empty and GeboortejaarmaandBiologischeMoeder is not: 136
The amount of rows where Geboortejaarmaand_relatie_ouder1 is not empty and GeboortejaarmaandBiologischeMoeder is empty: 4274
The amount of rows where Geboortejaarmaand_relatie_ouder1 is not empty and GeboortejaarmaandBiologischeMoeder is not empty and they are not equal: 9420
The amount of rows where Geboortejaarmaand_relatie_ouder1 and GeboortejaarmaandBiologischeMoeder are equal: 0


In [20]:
# Merge Geboortejaarmaand_relatie_ouder1 and GeboortejaarmaandBiologischeMoeder
schoolgaanden['Geboortejaarmaand_relatie_ouder1'] = schoolgaanden['Geboortejaarmaand_relatie_ouder1'].combine_first(schoolgaanden['GeboortejaarmaandBiologischeMoeder'])

#### 3.2 Migrants (CBS Indeling)

In [21]:
# Create substitution dictionary
migrants_cat = {'Syrië': 'Syrië',
                    'Nederland' : 'Nederland',
                                                                   'Maleisië': 'Overig Azië',
                                                                   'België': 'Europa (excl. NL)',
                                                                   'Polen': 'Europa (excl. NL)',
                                                                   'Verenigde Staten van Amerika': 'Overig Amerika en Oceanië',
                                                                   'Spanje': 'Europa (excl. NL)',
                                                                   'Portugal': 'Europa (excl. NL)',
                                                                   'Groot-Brittannië': 'Europa (excl. NL)',
                                                                   'Afghanistan': 'Overig Azië',
                                                                   'Oekraïne': 'Europa (excl. NL)',
                                                                   'Zuid-Afrika': 'Overig Afrika',
                                                                   'Denemarken': 'Europa (excl. NL)',
                                                                   'Canada': 'Overig Amerika en Oceanië',
                                                                   'Roemenië': 'Europa (excl. NL)',
                                                                   'Bondsrepubliek Duitsland': 'Europa (excl. NL)',
                                                                   'Ecuador': 'Overig Amerika en Oceanië',
                                                                   'Turkije': 'Turkije',
                                                                   'Jordanië': 'Overig Azië',
                                                                   'Israël': 'Overig Azië',
                                                                   'Libanon': 'Overig Azië',
                                                                   'Cyprus': 'Overig Azië', # Cyprus is geografisch gezien in Azië, maar politiek gezien in Europa
                                                                   'Brazilië': 'Overig Amerika en Oceanië',
                                                                   'Thailand': 'Overig Azië',
                                                                   'Singapore': 'Overig Azië',
                                                                   'Bulgarije': 'Europa (excl. NL)',
                                                                   'Zuid-Korea': 'Overig Azië',
                                                                   'Griekenland': 'Europa (excl. NL)',
                                                                   'China': 'Overig Azië',
                                                                   'Suriname': 'Suriname',
                                                                   'Chili': 'Overig Amerika en Oceanië',
                                                                   'Indonesië': 'Indonesië',
                                                                   'Marokko': 'Marokko',
                                                                   'Finland': 'Europa (excl. NL)',
                                                                   'Onbekend': np.nan,
                                                                   'Filipijnen': 'Overig Azië',
                                                                   'Armenië': 'Overig Azië',
                                                                   'Frankrijk': 'Europa (excl. NL)',
                                                                   'Iran': 'Overig Azië',
                                                                   'Hongarije': 'Europa (excl. NL)',
                                                                   'Dominicaanse Republiek': 'Overig Amerika en Oceanië',
                                                                   'India': 'Overig Azië',
                                                                   'Joegoslavië': 'Europa (excl. NL)',
                                                                   'Irak': 'Overig Azië',
                                                                   'Oezbekistan': 'Overig Azië',
                                                                   'Sri Lanka': 'Overig Azië',
                                                                   'Nederlandse Antillen': 'Nederlandse Cariben',
                                                                   'Sovjet-Unie': 'Europa (excl. NL)',
                                                                   'Litouwen': 'Europa (excl. NL)',
                                                                   'Ethiopië': 'Overig Afrika',
                                                                   'Korea': 'Overig Azië',
                                                                   'Soedan': 'Overig Afrika',
                                                                   'Somalië': 'Overig Afrika',
                                                                   'Saoedi-Arabië': 'Overig Azië',
                                                                   'Verenigde Arabische Emiraten': 'Overig Azië',
                                                                   'Australië': 'Overig Amerika en Oceanië',
                                                                   'Slowakije': 'Europa (excl. NL)',
                                                                   'Vietnam': 'Overig Azië',
                                                                   'Estland': 'Europa (excl. NL)',
                                                                   'Zaïre': 'Overig Afrika',
                                                                   'Zambia': 'Overig Afrika',
                                                                   'Noorwegen': 'Europa (excl. NL)',
                                                                   'Panama': 'Overig Amerika en Oceanië',
                                                                   'Peru': 'Overig Amerika en Oceanië',
                                                                   'Algerije': 'Overig Afrika',
                                                                   'Guatemala': 'Overig Amerika en Oceanië',
                                                                   'Zwitserland': 'Europa (excl. NL)',
                                                                   'Ghana': 'Overig Afrika',
                                                                   'Italië': 'Europa (excl. NL)',
                                                                   'Duitse Democratische Republiek': 'Europa (excl. NL)',
                                                                   'Eritrea': 'Overig Afrika',
                                                                   'Nigeria': 'Overig Afrika',
                                                                   'Colombia': 'Overig Amerika en Oceanië',
                                                                   'Macedonië': 'Europa (excl. NL)',
                                                                   'Egypte': 'Overig Afrika',
                                                                   'Pakistan': 'Overig Azië',
                                                                   'Nieuw-Zeeland': 'Overig Amerika en Oceanië',
                                                                   'Kameroen': 'Overig Afrika',
                                                                   'Guinee': 'Overig Afrika',
                                                                   'Sierra Leone': 'Overig Afrika',
                                                                   'Libië': 'Overig Afrika',
                                                                   'Aruba': 'Nederlandse Cariben',
                                                                   'Tsjecho-Slowakije': 'Europa (excl. NL)',
                                                                   'Azerbeidzjan': 'Overig Azië',
                                                                   'Uganda': 'Overig Afrika',
                                                                   'Democratische Republiek Congo': 'Overig Afrika',
                                                                   'nan': np.nan,
                                                                   'Burundi': 'Overig Afrika',
                                                                   'Gazastrook en Westelijke Jordaanoever': 'Overig Azië',
                                                                   'Gambia': 'Overig Afrika',
                                                                   'Angola': 'Overig Afrika',
                                                                   'Japan': 'Overig Azië',
                                                                   'Venezuela': 'Overig Amerika en Oceanië',
                                                                   'Kenya': 'Overig Afrika',
                                                                   'Federale Republiek Joegoslavië': 'Europa (excl. NL)'}

# Map all unique values to an overarching category
schoolgaanden['Geboorteland_cat'] = schoolgaanden['Geboorteland'].map(migrants_cat)
schoolgaanden['Geboorteland_relatie_ouder1_cat'] = schoolgaanden['Geboorteland_relatie_ouder1'].map(migrants_cat)
schoolgaanden['Geboorteland_relatie_ouder2_cat'] = schoolgaanden['Geboorteland_relatie_ouder2'].map(migrants_cat)

In [22]:
# Create substitution dictionary
migrants_cat3 = {'Syrië': 'Niet-Westers',
                    'Nederland': 'Nederlands',
                    'Marokko': 'Niet-Westers',
                    'Europa (excl. NL)': 'Westers',
                    'Turkije': 'Niet-Westers',
                    'Nederlandse Cariben': 'Niet-Westers',
                    'Overig Azië': 'Niet-Westers',
                    'Overig Amerika en Oceanië': 'Westers',
                    'Suriname': 'Niet-Westers',
                    'Overig Afrika': 'Niet-Westers',
                    'Indonesië': 'Niet-Westers'}

# Map all unique values to an overarching category
schoolgaanden['Geboorteland_cat3'] = schoolgaanden['Geboorteland_cat'].map(migrants_cat3)
schoolgaanden['Geboorteland_relatie_ouder1_cat3'] = schoolgaanden['Geboorteland_relatie_ouder1_cat'].map(migrants_cat3)
schoolgaanden['Geboorteland_relatie_ouder2_cat3'] = schoolgaanden['Geboorteland_relatie_ouder2_cat'].map(migrants_cat3)

In [23]:
schoolgaanden[['Geboorteland_cat3', 'Geboorteland_relatie_ouder1_cat3', 'Geboorteland_relatie_ouder2_cat3']].isna().mean()

Geboorteland_cat3                   0.096960
Geboorteland_relatie_ouder1_cat3    0.033174
Geboorteland_relatie_ouder2_cat3    0.049904
dtype: float64

In [24]:
# Create new column 'CBS_indeling'
schoolgaanden['CBS_indeling'] = schoolgaanden.apply(
    lambda x: (
        'Nederlands' if x['Geboorteland_cat3'] == 'Nederlands' and x['Geboorteland_relatie_ouder1_cat3'] == 'Nederlands' and x['Geboorteland_relatie_ouder2_cat3'] == 'Nederlands'
        else (
            'Kindmigrant' if x['Geboorteland_cat3'] == 'Nederlands' and (
                x['Geboorteland_relatie_ouder1_cat3'] in ['Westers', 'Niet-Westers'] or x['Geboorteland_relatie_ouder2_cat3'] in ['Westers', 'Niet-Westers']
            )
            else (
                'Migrant' if x['Geboorteland_cat3'] in ['Westers', 'Niet-Westers']
                else np.nan
            )
        )
    ), axis=1
)

In [25]:
#Count occurrences of CBS_indeling including missing values
schoolgaanden['CBS_indeling'].value_counts(dropna=False)

CBS_indeling
Nederlands     9524
Kindmigrant    2014
NaN            1835
Migrant         674
Name: count, dtype: int64

### 4. Create CoronIT unique keys

In [26]:
# Print the percentage of missing values per column
print(f'The percentage of missing values per column is:')
for col in schoolgaanden.columns:
    print(f'{col:<100}: {round(schoolgaanden[col].isnull().mean(), 2)}')

The percentage of missing values per column is:
Postcode                                                                                            : 0.0
Huisnummer                                                                                          : 0.0
Clientnummer                                                                                        : 0.0
B.M.R.                                                                                              : 0.0
DKTP                                                                                                : 0.0
DKTP-booster                                                                                        : 0.0
DTP                                                                                                 : 0.0
Hib                                                                                                 : 0.0
Pneu                                                                                                : 0.

In [27]:
# Remove twins from schoolgaanden
print(f'{len(schoolgaanden[schoolgaanden['Tweeling'] == True])} individual twins')
print(f'{len(schoolgaanden)} individuals in total')
schoolgaanden_without_twins = schoolgaanden[schoolgaanden['Tweeling'] == False][['Postcode', 'Huisnummer', 'Geboortejaarmaand_relatie_ouder1', 'Geslacht_relatie_ouder1', 'Geboortejaarmaand_relatie_ouder2', 'Geslacht_relatie_ouder2']]
schoolgaanden_twins = schoolgaanden[schoolgaanden['Tweeling'] == True][['Postcode', 'Huisnummer', 'Geboortejaarmaand_relatie_ouder1', 'Geslacht_relatie_ouder1', 'Geboortejaarmaand_relatie_ouder2', 'Geslacht_relatie_ouder2']]
schoolgaanden_twins = schoolgaanden_twins.drop_duplicates()
schoolgaanden_without_twins = pd.concat([schoolgaanden_without_twins, schoolgaanden_twins])
print(f'{len(schoolgaanden_without_twins)} individuals after removing twins')

285 individual twins
14047 individuals in total
12602 individuals after removing twins


In [28]:
# Get combinations representing first parents
parent_1 = schoolgaanden_without_twins[['Postcode', 'Huisnummer', 'Geboortejaarmaand_relatie_ouder1', 'Geslacht_relatie_ouder1']]
parent_1.columns = ['Postcode', 'Huisnummer', 'Geboortejaarmaand_relatie_ouder', 'Geslacht_relatie_ouder']

# Check
parent_1 = parent_1.dropna(subset=['Geboortejaarmaand_relatie_ouder'])
print(f'{len(parent_1)} first parents after deleting rows with missings')

12500 first parents after deleting rows with missings


In [29]:
# Get combinations representing second parents
parent_2 = schoolgaanden_without_twins[['Postcode', 'Huisnummer', 'Geboortejaarmaand_relatie_ouder2', 'Geslacht_relatie_ouder2']]
parent_2.columns = ['Postcode', 'Huisnummer', 'Geboortejaarmaand_relatie_ouder', 'Geslacht_relatie_ouder']

# Check
parent_2 = parent_2.dropna(subset=['Geboortejaarmaand_relatie_ouder'])
print(f'{len(parent_2)} second parents after deleting rows with missings')

12157 second parents after deleting rows with missings


In [30]:
# Stack the two dataframes, reset indices
parents = pd.concat([parent_1, parent_2]).reset_index(drop=True)
print(f'{len(parents)} parents in total')

# Show duplicate rows
print('-'*20)
print(f'{len(parents[parents.duplicated()])} duplicate parents')
parents = parents.drop_duplicates()
print(f'{len(parents)} parents after dropping duplicates')

24657 parents in total
--------------------
705 duplicate parents
23952 parents after dropping duplicates


In [31]:
# Print the percentages of missing values per column in parents
print(f'The percentage of missing values per column in parents is:')
for col in parents.columns:
    print(f'{col:<70}: {round(parents[col].isnull().mean(), 2)}')

The percentage of missing values per column in parents is:
Postcode                                                              : 0.0
Huisnummer                                                            : 0.0
Geboortejaarmaand_relatie_ouder                                       : 0.0
Geslacht_relatie_ouder                                                : 0.29


In [32]:
# Save parents as .csv
parents.to_csv('4_Data\CoronIT\keys\Ouders_schoolgaanden.csv', index=False)

### 5. Save .pkl's

In [33]:
# Save df_contactmomenten_og as a pickle
schoolgaanden.to_pickle('4_Data\Pickles\schoolgaanden.pkl')